# **Case Study #1 - Danny's Diner - Solutions**

## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>1. What is the total amount each customer spent at the restaurant?</b></span>

In [ ]:
SELECT
	s.customer_id,
	SUM(m.price) AS Total_spent
FROM sales s
JOIN menu m
	ON s.product_id = m.product_id
GROUP BY s.customer_id;

(3 row(s) affected)

Total execution time: 00:00:00.007

customer_id,total_spent
B,74
C,36
A,76


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>2. How many days has each customer visited the restaurant?</b></span>

In [ ]:
SELECT
	customer_id,
	COUNT(DISTINCT(order_date)) as no_of_days_visited
FROM sales
GROUP BY customer_id;

(3 row(s) affected)

Total execution time: 00:00:00.003

customer_id,no_of_days_visited
A,4
B,6
C,2


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>3. What was the first item from the menu purchased by each customer?</b></span>

In [ ]:
with cte AS (
SELECT
	
	s.customer_id,
	m.product_name,
	RANK() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS ranking
FROM sales s
JOIN menu m
	ON s.product_id = m.product_id)
SELECT
	customer_id,
	product_name
FROM cte
	WHERE ranking =1;

(5 row(s) affected)

Total execution time: 00:00:00.003

customer_id,product_name
A,curry
A,sushi
B,curry
C,ramen
C,ramen


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>4. What is the most purchased item on the menu and how many times was it purchased by all customers?</b></span>

In [ ]:
SELECT
	s.customer_id,
	m.product_name,
	COUNT(*) as no_of_times
FROM sales s
JOIN menu m
	ON s.product_id = m.product_id
	where m.product_name =
	(SELECT
		m.product_name
	FROM sales s
	JOIN menu m
		ON s.product_id = m.product_id
    GROUP BY m.product_name
    ORDER BY count(*) DESC
    LIMIT 1)
GROUP BY s.customer_id,m.product_name;

(3 row(s) affected)

Total execution time: 00:00:00.002

customer_id,product_name,no_of_times
B,ramen,2
A,ramen,3
C,ramen,3


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>5. Which item was the most popular for each customer?</b></span>

In [ ]:
with cte2 as (WITH cte AS (
	SELECT
		s.customer_id,
    		m.product_name,
    		count(m.product_name) as popular_item_count
	FROM sales s
	JOIN menu m 
		ON s.product_id = m.product_id
    	GROUP BY s.customer_id,m.product_name)
    SELECT 
    customer_id,
    product_name,
    popular_item_count,
    DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY popular_item_count DESC) as rankings
    FROM cte)
    SELECT
    	customer_id,
        product_name,
        popular_item_count
    FROM cte2 WHERE rankings =1;

(5 row(s) affected)

Total execution time: 00:00:00.003

customer_id,product_name,popular_item_count
A,ramen,3
B,sushi,2
B,curry,2
B,ramen,2
C,ramen,3


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>6. Which item was purchased first by the customer after they became a member?</b></span>

In [ ]:
WITH cte as(SELECT
	me.customer_id,
    me.join_date,
    s.order_date,
    m.product_name,
    RANK() OVER (PARTITION BY me.customer_id order by s.order_date) as rankings
 FROM members me
 JOIN sales s
	ON me.customer_id = s.customer_id
 JOIN menu m
	ON s.product_id = m.product_id
    WHERE s.order_date >= me.join_date)
    SELECT customer_id,product_name FROM cte where rankings =1 ;

(2 row(s) affected)

Total execution time: 00:00:00.001

customer_id,product_name
A,curry
B,sushi


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit;"><b>7. Which item was purchased just before the customer became a member?</b></span>

In [ ]:
WITH cte as(SELECT
	me.customer_id,
    me.join_date,
    s.order_date,
    m.product_name,
    RANK() OVER (PARTITION BY me.customer_id order by s.order_date desc) as rankings
 FROM members me
 JOIN sales s
	ON me.customer_id = s.customer_id
 JOIN menu m
	ON s.product_id = m.product_id
    WHERE s.order_date < me.join_date)
    SELECT customer_id,product_name FROM cte where rankings =1 ;

(3 row(s) affected)

Total execution time: 00:00:00.003

customer_id,product_name
A,sushi
A,curry
B,sushi


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>8. What is the total items and amount spent for each member before they became a member?</b></span>

In [ ]:
SELECT
 	me.customer_id,
	COUNT(m.product_id) as total_items,
	SUM(m.price) as amount_spent
FROM members me
JOIN sales s
	ON me.customer_id = s.customer_id 
JOIN menu m
	ON s.product_id = m.product_id
WHERE s.order_date < me.join_date
        GROUP BY me.customer_id;

(2 row(s) affected)

Total execution time: 00:00:00.003

customer_id,total_items,amount_spent
A,2,25
B,3,40


## <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; background-color: rgb(255, 255, 255); color: rgb(64, 64, 64);"><b>9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?</b></span>

In [ ]:
SELECT
	s.customer_id,
	SUM(case
		when m.product_name = 'sushi' then m.price*20
        	else price*10
	end) as points_earned
FROM sales s
JOIN menu m
	ON s.product_id = m.product_id
GROUP BY s.customer_id
ORDER BY s.customer_id;

(3 row(s) affected)

Total execution time: 00:00:00.002

customer_id,points_earned
A,860
B,940
C,360


## <span style="background-color: rgb(255, 255, 255); color: rgb(64, 64, 64); font-family: inherit; font-size: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit;"><b>10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?</b></span>

In [ ]:

SELECT
	me.customer_id,
	SUM(m.price*20)  as points_earned_in_first_week
FROM members me
JOIN sales s
	ON me.customer_id = s.customer_id
JOIN menu m
	ON s.product_id = m.product_id
WHERE s.order_date between me.join_date and me.join_date + INTERVAL '7 day'
GROUP BY me.customer_id
ORDER BY me.customer_id;

(2 row(s) affected)

Total execution time: 00:00:00.004

customer_id,points_earned_in_first_week
A,1020
B,440


# **Bonus Questions**

## **Join All The Things**

In [ ]:
SELECT
	s.customer_id,
	s.order_date,
    	m.product_name,
    	m.price,
    	CASE
		WHEN s.order_date < me.join_date THEN 'N'
       		WHEN me.join_date is null THEN 'N'
        	ELSE 'Y'
	END as member
FROM sales s
JOIN menu m
	ON s.product_id = m.product_id
LEFT JOIN members me
	ON s.customer_id = me.customer_id;

(15 row(s) affected)

Total execution time: 00:00:00.004

customer_id,order_date,product_name,price,member
B,2021-01-04,sushi,10,N
A,2021-01-01,sushi,10,N
B,2021-01-11,sushi,10,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-10,ramen,12,Y


## **Rank All The Things**

In [ ]:
WITH cte as(
	SELECT
		s.customer_id,
    		s.order_date,
    		m.product_name,
    		m.price,
    		CASE
			WHEN s.order_date < me.join_date THEN 'N'
        		WHEN me.join_date is null THEN 'N'
        		ELSE 'Y'
		END as member
	FROM sales s
	JOIN menu m
		ON s.product_id = m.product_id
	LEFT JOIN members me
		ON s.customer_id = me.customer_id)
    SELECT
    *,
    CASE WHEN member = 'Y' THEN rank() OVER (PARTITION BY customer_id, member ORDER BY order_date) ELSE NULL END as ranking
    FROM cte

(15 row(s) affected)

Total execution time: 00:00:00.003

customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,NULL
A,2021-01-01,curry,15,N,NULL
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,NULL
B,2021-01-02,curry,15,N,NULL
B,2021-01-04,sushi,10,N,NULL
B,2021-01-11,sushi,10,Y,1
